In [1]:
import pandas as pd
import numpy as np
import pickle
import utils
from collections import Counter

In [2]:
data = pd.read_csv('challenge_dat.csv')

In [3]:
data.head(10)

,RESTAURANT_NAME,CITY,COUNTRY,R_PRIMARY_LOC_TAG,R_OTHER_LOC_TAG,R_MENU_CATEGORY,R_OTHER_MENU_TAGS,ITEM_NAME,DESCRIPTION,IS_ALCOHOL
0,Hero Sandwich,Hong Kong,Hong Kong,american,western,Sandwiches,"american, western, burgers, Pasta, Smoothies, ...",Corn Beef Baked Potato,鹹牛肉焗薯,False
1,Hamachi-Ya,Sydney,Australia,japanese,NaN,japanese,"sushi, bento, Cream of the crop, Big flavours,...",Spicy Garlic Edamame,Poached green soy beans tossed with salt fresh...,False
2,Hamachi-Ya,Sydney,Australia,japanese,NaN,japanese,"sushi, bento, Cream of the crop, Big flavours,...",Soft Shell Crab (2pcs),Deep-fried soft shell crab with light ponzu sauce,False
3,Hamachi-Ya,Sydney,Australia,japanese,NaN,japanese,"sushi, bento, Cream of the crop, Big flavours,...",Harumaki (4Pcs),Deep-fried Japanese style vegetable springroll,False
4,Hamachi-Ya,Sydney,Australia,japanese,NaN,japanese,"sushi, bento, Cream of the crop, Big flavours,...",Avocado,NaN,False
5,Lion City Kitchen,London,UK,american,Singaporean,american,"Singaporean, burgers, Guilty pleasures, tacos,...",Rendang Beef,Six-hour slow cooked beef brisket and shin in ...,False
6,Céleste,Paris,France,asian,chinese,asian,chinese,Bo bun boeuf,Vermicelle au boeuf et nems,False
7,Céleste,Paris,France,asian,chinese,asian,chinese,Crevettes au curry rouge,Et à la crème de coco,False
8,Céleste,Paris,France,asian,chinese,asian,chinese,Emincé de bœuf mariné,A la façon du chef,False
9,Céleste,Paris,France,asian,chinese,asian,chinese,Nems au boeuf,2 pièces,False


In [4]:
ascii_data = data.copy()
#remove non ascii from description, and text items
columns_to_ascii = ['R_OTHER_MENU_TAGS','ITEM_NAME','DESCRIPTION']

In [5]:
ascii_data = ascii_data.dropna(subset=['R_OTHER_MENU_TAGS','DESCRIPTION','ITEM_NAME','R_MENU_CATEGORY','R_PRIMARY_LOC_TAG'])
print(len(ascii_data), ' after na removal')
for col in columns_to_ascii:
    ascii_data[col] = ascii_data[col].astype(str).apply(lambda x: ''.join(['' if ord(i) < 32 or ord(i) > 126 else i for i in x]))
    ascii_data[col].replace('', np.nan, inplace=True)
#remove nan
df = ascii_data.dropna(subset=['R_OTHER_MENU_TAGS','DESCRIPTION','ITEM_NAME','R_MENU_CATEGORY','R_PRIMARY_LOC_TAG'])
print(len(df), ' after ascii')

775109  after na removal
738849  after ascii


In [6]:
#merging the 3 columns
df['food'] = df['R_OTHER_MENU_TAGS'] + "," + df['ITEM_NAME'] + "," + df['DESCRIPTION']
df = df.drop(columns_to_ascii, axis=1)

/Users/aimihat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
df = df.loc[:10000]
tok_df = df.copy()
tok_df['food'] = tok_df['food'].apply(utils.tokenize)

In [12]:
#print(df[df['food'].str.contains('13th')]['food'].tolist())

In [15]:
all_words = []
for row in tok_df['food']:
    for word in row.split(','):
        all_words.append(word)
        
len(set(all_words))

5583

In [17]:
#tok_df['food'].to_csv('10000.csv')

In [10]:
Counter(all_words).most_common(100)

[('for', 1222),
 ('with', 1170),
 ('and', 1117),
 ('lunch', 842),
 ('salad', 747),
 ('onli', 744),
 ('deliveroo', 738),
 ('great', 710),
 ('chicken', 572),
 ('flavour', 560),
 ('the', 546),
 ('fish', 538),
 ('small', 521),
 ('big', 510),
 ('price', 505),
 ('pizza', 489),
 ('sauc', 467),
 ('perfect', 448),
 ('share', 446),
 ('cream', 441),
 ('food', 421),
 ('free', 402),
 ('curri', 397),
 ('pasta', 392),
 ('healthi', 382),
 ('crop', 373),
 ('sushi', 347),
 ('breakfast', 345),
 ('gem', 330),
 ('local', 329),
 ('fri', 327),
 ('bbq', 324),
 ('guilti', 322),
 ('pleasur', 322),
 ('tomato', 280),
 ('vegetarian', 267),
 ('asian', 267),
 ('ticket', 267),
 ('restaur', 267),
 ('sweet', 265),
 ('burger', 255),
 ('onion', 248),
 ('gluten', 242),
 ('chees', 224),
 ('serv', 224),
 ('tandoori', 213),
 ('drink', 211),
 ('indian', 209),
 ('mit', 205),
 ('und', 199),
 ('cook', 197),
 ('famili', 195),
 ('italian', 194),
 ('favourit', 192),
 ('mezz', 192),
 ('seafood', 190),
 ('grill', 188),
 ('fresh', 183

In [20]:
#remove unwanted words
#generic_words = ''
#all_words.remove(generic_words)

In [ ]:
#for everyword create column with 1 if appears 0 else